In [1]:
pip install pysindy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 2.2 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import random
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pysindy as ps
from sklearn.preprocessing import StandardScaler
from dynamics_generator_1d import saddle_bif, trans_bif, super_pitchfork, sub_pitchfork, Hysteresis, gen_BIF

#1. Data Processing

##1.1 Generating Data for Bifurcations

In [4]:
saddle_gen = gen_BIF(N = 1000, func = saddle_bif, noise = None)
saddle_gen_g = gen_BIF(N = 1000, func = saddle_bif, noise = 'Gaussian')
trans_gen = gen_BIF(N = 1000, func = trans_bif, noise = None)
trans_gen_g = gen_BIF(N = 1000, func = trans_bif, noise = 'Gaussian')
super_gen = gen_BIF(N = 1000, func = super_pitchfork, noise = None)
super_gen_g = gen_BIF(N = 1000, func = super_pitchfork, noise = 'Gaussian')
sub_gen = gen_BIF(N = 1000, func = sub_pitchfork, noise = None)
sub_gen_g = gen_BIF(N = 1000, func = sub_pitchfork, noise = 'Gaussian')
hyst_gen = gen_BIF(N = 1000, func = Hysteresis, noise = None)
hyst_gen_g = gen_BIF(N = 1000, func = Hysteresis, noise = 'Gaussian')
Data_pos = np.concatenate([saddle_gen, saddle_gen_g, trans_gen,trans_gen_g,super_gen,
                       super_gen_g, sub_gen, sub_gen_g, hyst_gen, hyst_gen_g], axis = 0)

In [5]:
saddle_gen_ = gen_BIF(N = 1000, func = saddle_bif, noise = None, sign='neg')
saddle_gen_g_ = gen_BIF(N = 1000, func = saddle_bif, noise = 'Gaussian', sign='neg')
trans_gen_ = gen_BIF(N = 1000, func = trans_bif, noise = None, sign='neg')
trans_gen_g_ = gen_BIF(N = 1000, func = trans_bif, noise = 'Gaussian', sign='neg')
super_gen_ = gen_BIF(N = 1000, func = super_pitchfork, noise = None, sign='neg')
super_gen_g_ = gen_BIF(N = 1000, func = super_pitchfork, noise = 'Gaussian', sign='neg')
sub_gen_ = gen_BIF(N = 1000, func = sub_pitchfork, noise = None, sign='neg')
sub_gen_g_ = gen_BIF(N = 1000, func = sub_pitchfork, noise = 'Gaussian', sign='neg')
hyst_gen_ = gen_BIF(N = 1000, func = Hysteresis, noise = None, sign='neg')
hyst_gen_g_ = gen_BIF(N = 1000, func = Hysteresis, noise = 'Gaussian', sign='neg')
Data_neg = np.concatenate([saddle_gen_, saddle_gen_g_, trans_gen_,trans_gen_g_,super_gen_,
                       super_gen_g_, sub_gen_, sub_gen_g_, hyst_gen_, hyst_gen_g_], axis = 0)

In [6]:
Data = np.concatenate([Data_pos, Data_neg], axis = 0)
np.random.shuffle(Data)

In [7]:
X_data, y_data = [], []
library = ps.PolynomialLibrary(degree = 4)
for series, time, label in Data:
  try:
    model_sindy = ps.SINDy(feature_library = library)
    model_sindy.fit(x = series, t = time)
    coef = model_sindy.coefficients().reshape(-1)
    X_data.append(coef)
    y_data.append(label)
  except:
    pass

/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:191: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coef

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, stratify = y_data, test_size = 0.1, random_state = 42)

#2. Training Process
## 2.1  Support Vector Machine

### 2.1.1 Linear SVM

In [9]:
from sklearn.svm import LinearSVC, SVC

lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train,y_train)

from sklearn.metrics import accuracy_score
y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.4051666666666667

In [10]:
from sklearn.metrics import confusion_matrix    
y_pred = lin_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))                    
print(confusion_matrix(y_test, y_pred) )

0.411
[[ 96  78  24  64 138]
 [ 45  55  30 154 116]
 [  0   3 132  69 196]
 [ 35   7  82 233  43]
 [  0   0  94   0 306]]


### 2.1.2 RBM SVM

In [11]:
rbf_clf = SVC(kernel = 'rbf', gamma = 0.7, C=1, max_iter = 2000)    

rbf_clf.fit(X_train,y_train)
y_pred = rbf_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.9408888888888889

In [12]:
y_pred = rbf_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))                   
print(confusion_matrix(y_test, y_pred) )

0.821
[[388   2   0  10   0]
 [ 84 291   3  22   0]
 [ 14  10 336  40   0]
 [140   4  24 232   0]
 [  4   1   0   0 395]]


In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## 2.2 Logistic Regression

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score


classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.4998888888888889

In [15]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.498

## 2.3 KNN

In [16]:
from sklearn.metrics import confusion_matrix, accuracy_score

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.8417777777777777

In [17]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.7765

# 2.4 Naive Bayes

In [18]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.2795

In [19]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.2815

## 2.5 Decision Tree

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.9378888888888889

In [21]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.864

## 2.6 Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.9182777777777777

In [23]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.8715